In [1]:
import time
#import cupy as cu
import numpy as np
import pandas as pa
import itertools
from pymopt.utils import _deprecate_positional_args
#import readDICOM as dcm
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("ticks", {'grid.linestyle': '--'})
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
import os
workspace = os.getcwd()
def calTime(end, start):
    elapsed_time = end - start
    q, mod = divmod(elapsed_time, 60)
    if q < 60:
        print('Calculation time: %d minutes %d seconds.' % (q, mod))
    else:
        q2, mod2 = divmod(q, 60)
        print('Calculation time: %d h %d minutes.' % (q2, mod2))

In [2]:
class MonteCalro:
    def __init__(self):
        self.nPh = 1000
        self.f_bit = 'float32'
        self.vectorTh = 0.99999
        
        self.v_result = np.empty((3,1)).astype(self.f_bit)
        self.p_result = np.empty((3,1)).astype(self.f_bit)
        self.add_result = np.empty((3,1)).astype(self.f_bit)
        self.w_result = np.empty(1).astype(self.f_bit)
        
        self.p = np.empty((3,1)).astype(self.f_bit)
        self.v = np.empty((3,1)).astype(self.f_bit)
        self.w = np.empty(1).astype(self.f_bit)
    
    def start(self):
        print("")
        print("###### Start ######")
        print("")
        
        start_ = time.time()
        
        count = self.monteCycle(start_)
        self.endProcess()

        #結果の表示
        print("")
        print("###### Finish ######")
        print("Maximum step number: %s"%count)
        self.getRdTtRate()
        self.calTime(time.time(), start_)
        return self
        
    def monteCycle(self,start_):
        count = 0
        counter = 2
        w_size = 1
        # Let's MonteCalro!
        while w_size != 0:
            w_size = self.stepMovement()
            count+=1
            if count%counter==0:
                counter*=2
                print("Progress: %s [％]"%round((1-w_size/self.nPh)*100,3))
                self.calTime(time.time(), start_)
                print()
        return count
        
    
    def endProcess(self):
        pass
    
    def vectorUpdate(self,v,G):
        index = np.where(G==0.0)[0]
        cosTh = np.empty_like(G)
        if list(index) != []:
            rand_ = np.random.rand(G.size).astype(self.f_bit)
            cosTh[index] = 2*rand_[index]-1
            index = np.where(G!=0)[0]
            if list(index) != []:
                cosTh[index] = ((1+G[index]**2\
                                 -((1-G[index]**2)/(1-G[index]+2*G[index]*rand_[index]))**2)/(2*G[index]))
        else:
            cosTh = (1+G**2-((1-G**2)/(1-G+2*G*np.random.rand(G.size).astype(self.f_bit)))**2)/(2*G)
        sinTh = np.sqrt(1-cosTh**2)

        #cos(fai)とsin(fai)と求める
        Fi = 2*np.pi*np.random.rand(G.size).astype(self.f_bit)
        cosFi = np.cos(Fi)
        sinFi = np.sin(Fi)

        #Zが１かそれ以外で分離
        th = self.vectorTh
        v1_index = np.where(np.abs(v[2])<=th)[0]
        v2_index = np.where(np.abs(v[2])>th)[0]

        #Z方向ベクトルが0.99999以下
        cosTh1 = cosTh[v1_index]; sinTh1 = sinTh[v1_index]
        cosFi1 = cosFi[v1_index]; sinFi1 = sinFi[v1_index]
        v1 = v[:,v1_index]
        B = np.sqrt(1-v1[2]**2)
        A = np.array([
            sinTh1*(v1[0]*v1[2]*cosFi1-v1[1]*sinFi1)/B,
            sinTh1*(v1[1]*v1[2]*cosFi1+v1[0]*sinFi1)/B,
            -sinTh1*cosFi1*B,
        ])
        v[:,v1_index] = A+v1*cosTh1

        #Z方向ベクトルが0.99999以上
        cosTh2 = cosTh[v2_index]; sinTh2 = sinTh[v2_index]
        cosFi2 = cosFi[v2_index]; sinFi2 = sinFi[v2_index]
        v[:,v2_index] = np.array([
            sinTh2*cosFi2,
            sinTh2*sinFi2,
            np.sign(v[2,v2_index])*cosTh2,
        ],dtype=self.f_bit)

        v = v/np.linalg.norm(v,axis=0)
        return v

    def positionUpdate(self,p,v,L):
        return p+v*L

    #光子の１ステップにおけるエネルギーの損失を計算
    def wUpdate(self,w,ma,mt,rato,p):
        dw = w*rato*ma/mt
        if self.fluence != False:
            self.fluence.saveFluesnce(p,dw)
        return w-dw
            
    def russianRoulette(self,w):
        ## 確率的に光子を生き返らせます。
        m = 10
        ra = np.random.rand(w.size).astype(self.f_bit)
        index = np.where(ra>(1/m))[0].tolist()
        w[index] = 0
        index = np.where(ra<=(1/m))[0].tolist()
        w[index] = w[index]*m
        return w

    #光子の移動距離, uniformly distributed over the interval (0,1)
    def stepLength(self,size):
        return -np.log(np.random.rand(size)).astype(self.f_bit)
    
    #任意の位置(indexの行)が１でそれ以外は0の行列を作る
    def create01Array(self,index,m=3):
        n = index.size
        array_0_1 = np.zeros(m*n,dtype = bool)
        array_0_1[index+m*np.arange(n)] = 1
        return array_0_1.reshape(n,m).T
    
    @classmethod
    def _get_param_names(cls):
        """Get parameter names for the estimator"""
        # fetch the constructor or the original constructor before
        # deprecation wrapping if any
        init = getattr(cls.__init__, 'deprecated_original', cls.__init__)
        if init is object.__init__:
            # No explicit constructor to introspect
            return []

        # introspect the constructor arguments to find the model parameters
        # to represent
        init_signature = inspect.signature(init)
        # Consider the constructor parameters excluding 'self'
        parameters = [p for p in init_signature.parameters.values()
                      if p.name != 'self' and p.kind != p.VAR_KEYWORD]
        for p in parameters:
            if p.kind == p.VAR_POSITIONAL:
                raise RuntimeError("monte method should always "
                                   "specify their parameters in the signature"
                                   " of their __init__ (no varargs)."
                                   " %s with constructor %s doesn't "
                                   " follow this convention."
                                   % (cls, init_signature))
        # Extract and sort argument names excluding 'self'
        return sorted([p.name for p in parameters])
    
    def set_params(self,**params):
        if not params:
            # Simple optimization to gain speed (inspect is slow)
            return self
        valid_params = self.get_params(deep=True)

        nested_params = defaultdict(dict)  # grouped by prefix
        for key, value in params.items():
            key, delim, sub_key = key.partition('__')
            if key not in valid_params:
                raise ValueError('Invalid parameter %s for estimator %s. '
                                 'Check the list of available parameters '
                                 'with `monte.get_params().keys()`.' %
                                 (key, self))

            if delim:
                nested_params[key][sub_key] = value
            else:
                setattr(self, key, value)
                valid_params[key] = value

        for key, sub_params in nested_params.items():
            valid_params[key].set_params(**sub_params)

        return self
    
    def get_params(self, deep=True):
        
        out = dict()
        for key in self._get_param_names():
            try:
                value = getattr(self, key)
            except AttributeError:
                warnings.warn('From version 0.24, get_params will raise an '
                              'AttributeError if a parameter cannot be '
                              'retrieved as an instance attribute. Previously '
                              'it would return None.',
                              FutureWarning)
                value = None
            if deep and hasattr(value, 'get_params'):
                deep_items = value.get_params().items()
                out.update((key + '__' + k, val) for k, val in deep_items)
            out[key] = value
        return out
    
    def calTime(self, end, start):
        elapsed_time = end - start
        q, mod = divmod(elapsed_time, 60)
        if q < 60:
            print('Calculation time: %d minutes %d seconds.' % (q, mod))
        else:
            q2, mod2 = divmod(q, 60)
            print('Calculation time: %d h %d minutes.' % (q2, mod2))
            
    def getRdTtRate(self):
        Tt_index = np.where(self.v_result[2]>0)[0]
        Rd_index = np.where(self.v_result[2]<0)[0]
        print('######')
        print('Mean Rd %0.6f'%(self.w_result[Rd_index].sum()/self.nPh))
        print('Mean Tt %0.6f'%(self.w_result[Tt_index].sum()/self.nPh))
        print()

In [3]:
class VoxcelModel:
        
    def built(self):
        pass
    def getAbsorptionCoeff(self,add):
        x,y,z = add
        index = self.voxcel_model[x,y,z]
        return self.ma[index]
    def getScatteringCoeff(self,add):
        x,y,z = add
        index = self.voxcel_model[x,y,z]
        return self.ms[index]
    def getAnisotropyCoeff(self,add):
        x,y,z = add
        index = self.voxcel_model[x,y,z]
        return self.g[index]
    def getReflectiveIndex(self,add):
        x,y,z = add
        index = self.voxcel_model[x,y,z]+1
        return self.n[index]
    
class VoxcelPlateModel(VoxcelModel):
        
    def built(self,*,thickness,xy_size,space,ma,ms,g,n_list,n_air,f = 'float32'):
        #-1はモデルの外側
        self.voxcel_space = space
        thickness = [0]+thickness
        self.xy_size = xy_size
        b = 0; b_list = []
        for i in  thickness:
            b += i
            b_list.append(b)
        
        borderposit = np.array(b_list).astype(f)
        nxy_box = np.round(self.xy_size/space).astype(int)
        nz_box = np.round(borderposit/space).astype(int)
        self.voxcel_model = np.zeros((nxy_box,nxy_box,nz_box[-1]),dtype = 'int8')
        for i in range(nz_box.size-1):
            self.voxcel_model[:,:,nz_box[i]:nz_box[i+1]] = i
        
        A = np.ones((nxy_box,nxy_box),dtype='int8')*-1
        self.voxcel_model = np.concatenate([self.voxcel_model, np.array([A]).T],2)
        self.voxcel_model = np.concatenate([np.array([A]).T,self.voxcel_model],2)
        A = np.ones((nxy_box,nz_box[-1]+2),dtype='int8')*-1
        self.voxcel_model = np.concatenate([self.voxcel_model, np.array([A])])
        self.voxcel_model = np.concatenate([np.array([A]),self.voxcel_model])        
        print("%d Mbyte" % (self.voxcel_model.nbytes*1e-6))
        
        self.n =np.array([n_air]+n_list).astype(f)
        self.ms = np.array(ms).astype(f)
        self.ma = np.array(ma).astype(f)
        self.g = np.array(g).astype(f)
        

In [20]:
        
class VoxcelMonteCalro(MonteCalro):
    #@_deprecate_positional_args
    def __init__(self,*,nPh,model,fluence = False,f_bit='float32'):
        super().__init__()
        self.f_bit = f_bit
        self.nPh = nPh
        self.model = model
        self.fluence = fluence
        self.generateInisalCoodinate(self.nPh)
        
    def generateInisalCoodinate(self,nPh,f = 'float32'):
        center_add_xy = int(self.model.voxcel_model.shape[0]/2)
        self.add =  np.full((3, nPh),center_add_xy).astype("int16")
        self.add[2] = 1
        self.p = np.zeros((3,nPh)).astype(f)
        self.p[2] = -self.model.voxcel_space/2
        self.v = np.zeros((3,nPh)).astype(f)
        self.v[2] = 1
        self.w = np.ones(nPh).astype(f)
        self.w = self.initialWeight(self.w)
        
    def encooder(self):
        space = model.voxcel_space
        center_add_xy = int(self.model.voxcel_model.shape[0]/2)
        
        
    def initialWeight(self,w):
        Rsp = 0
        n1 = self.model.n[0]
        n2 = self.model.n[1]
        if n1 != n2:
            Rsp = ((n1-n2)/(n1+n2))**2
        return w-Rsp
    
    @_deprecate_positional_args   
    def built(self,**kwargs):
        self.model.built(**kwargs)
        self.generateInisalCoodinate(self.nPh)
    
    def photonVanishing(self,p,v,w,add):
        #photn waight が0.0001以下
        del_index = np.where(w<=0.0001)[0]
        if list(del_index) != []:
            w[del_index] = self.russianRoulette(w[del_index])
            del_index = np.where(w==0)[0]
            v = np.delete(v, del_index, axis = 1)
            p = np.delete(p, del_index, axis = 1)
            w = np.delete(w, del_index)
            add = np.delete(add,del_index, axis = 1)
        return p,v,w,add
    
    def borderOut(self,p,v,w,add,index):
        self.v_result = np.concatenate([self.v_result, v[:,index]],axis = 1)
        self.p_result = np.concatenate([self.p_result, p[:,index]],axis = 1)
        self.add_result = np.concatenate([self.add_result, add[:,index]],axis = 1)
        self.w_result = np.concatenate([self.w_result,w[index]])

        p = np.delete(p,index, axis = 1)
        v = np.delete(v,index, axis = 1)
        add = np.delete(add,index, axis = 1)
        
        w = np.delete(w,index)
        return p,v,w,add
    
    def borderAct(self,v,p,add,s,db):
        l = self.model.voxcel_space
        A = self.create01Array(np.argmin(db,axis=0),m=3)
        B = ~A
        ni = self.model.getReflectiveIndex(add)
        sigAv = A*np.sign(v).astype("int16")
        ds = ((l/2-p*np.sign(v))/abs(v)).min(0)
        
        s -= ds
        p = sigAv*l/2+B*(p+v*ds)
        nt = self.model.getReflectiveIndex((add+sigAv))
        pb_index = np.where(ni!=nt)[0]
        pn_index = np.where(ni==nt)[0]
        
        if list(pb_index) != []:
            p[:,pn_index] = -sigAv[:,pn_index]*l/2
            add[:,pn_index] += sigAv[:,pn_index]
            #透過判別
            #入射角の計算
            ai = np.abs(A[:,pb_index]*v[:,pb_index])
            ai = np.arccos(abs(ai[ai!=0]))
            sub_index = np.where((ai>np.arcsin(nt[pb_index]/ni[pb_index])))[0]
            
            #屈折角の計算
            ##(ai,at)=(0,0)の時は透過させ方向ベクトルは変化させない
            at = np.arcsin(ni[pb_index]*np.sin(ai)/nt[pb_index])
            ind = np.where(ai!=0)[0]
            Ra = np.random.rand(ai.size)
            Ra[ind] = -((np.sin(ai[ind]-at[ind])/np.sin(ai[ind]+at[ind]))**2\
              +(np.tan(ai[ind]-at[ind])/np.tan(ai[ind]+at[ind]))**2)/2
            
            #全反射はRaを強制的に0以下にし反射させる
            Ra[sub_index] = -1
            #透過と反射のindexを取得
            vl_index = pb_index[np.where(Ra<=0)[0]]#反射
            sub_index = np.where(Ra>0)[0]
            vt_index = pb_index[sub_index]#透過
            at = at[sub_index]
            v[:,vl_index] = B[:,vl_index]*v[:,vl_index]-A[:,vl_index]*v[:,vl_index]
            v[:,vt_index] = (ni[vt_index]/nt[vt_index])*B[:,vt_index]*v[:,vt_index]\
            +np.sign(sigAv[:,vt_index])*np.cos(at)
            add[:,vt_index] += sigAv[:,vt_index]
            p[:,vt_index] = -sigAv[:,vt_index]*l/2
        else:
            p = -sigAv*l/2
            add += sigAv
        return v,p,add,s
    
    def RTInterface(self,v,p,add,s):
        box_model = self.model.voxcel_model
        l = self.model.voxcel_space
        v_,p_,add_,s_ = v,p,add,s
        index = np.arange(s.size)
        out_index = []
        pb_index = [0]
        while True:
            db = (l/2-p_*np.sign(v_))/np.abs(v_)
            ma = self.model.getAbsorptionCoeff(add_)
            ms = self.model.getScatteringCoeff(add_)
            mt = ma+ms
            s_ = s_/mt
            pb_index = np.where(s_-db.min(0)>=0)[0]
            pn_index = np.delete(np.arange(s_.size),pb_index)
            if list(pb_index) != []:
                v[:,index[pn_index]] = v_[:,pn_index]
                p[:,index[pn_index]] = p_[:,pn_index]\
                +s_[pn_index]*v_[:,pn_index]
                add[:,index[pn_index]] = add_[:,pn_index]
                index = index[pb_index]
                v_,p_,add_,s_ = self.borderAct(v_[:,pb_index],
                                               p_[:,pb_index],
                                               add_[:,pb_index],
                                               s_[pb_index],
                                               db[:,pb_index])
                s_ = s_*mt[pb_index]
                out_index_ = np.where(box_model[add_[0],add_[1],add_[2]] < 0)[0]
                
                if list(out_index_) != []:
                    
                    v[:,index[out_index_]] = v_[:,out_index_]
                    p[:,index[out_index_]] = p_[:,out_index_]
                    add[:,index[out_index_]] = add_[:,out_index_]
                    out_index += list(index[out_index_])
                    v_ = np.delete(v_,out_index_, axis = 1)
                    p_ = np.delete(p_,out_index_, axis = 1)
                    add_ = np.delete(add_,out_index_, axis = 1)
                    s_ = np.delete(s_,out_index_)
            else:
                v = v_
                p = p_+s_*v_
                break
        return v,p,add,out_index
    
    def stepMovement(self):
        l = self.model.voxcel_space
        
        p,v,w = self.p,self.v,self.w
        add = self.add
        s = np.random.rand(w.size)
        v,p,add,index = self.RTInterface(v,p,add,s)
        p,v,w,add = self.borderOut(p,v,w,add,index)
        G = self.model.getAnisotropyCoeff(add)
        v = self.vectorUpdate(v,G)
        ma = self.model.getAbsorptionCoeff(add)
        ms = self.model.getScatteringCoeff(add)
        w = self.wUpdate(w,ma,ma+ms,1,p)
        self.p,self.v,self.w,self.add = self.photonVanishing(p,v,w,add)
        return self.w.size

In [21]:
params = {
    'thickness':[0.1,0.1,0.2],
    'xy_size':10,
    'space':0.01,
    'ma':[1,1,2],
    'ms':[100,10,10],
    'g':[0.9,0,0.7],
    'n_list':[1.37,1.37,1.37],
    'n_air':1.,
}

model = VoxcelPlateModel()
model.built(**params)

monte = VoxcelMonteCalro(nPh = 100,model = model)



42 Mbyte


In [505]:
model.voxcel_space

0.01

In [519]:
model.voxcel_model.shape

(1002, 1000, 42)

In [7]:
monte.w_result

array([nan], dtype=float32)

In [516]:
monte.w

array([0.9756271, 0.9756271, 0.9756271, 0.9756271, 0.9756271, 0.9756271,
       0.9756271, 0.9756271, 0.9756271, 0.9756271, 0.9756271, 0.9756271,
       0.9756271, 0.9756271, 0.9756271, 0.9756271, 0.9756271, 0.9756271,
       0.9756271, 0.9756271, 0.9756271, 0.9756271, 0.9756271, 0.9756271,
       0.9756271, 0.9756271, 0.9756271, 0.9756271, 0.9756271, 0.9756271,
       0.9756271, 0.9756271, 0.9756271, 0.9756271, 0.9756271, 0.9756271,
       0.9756271, 0.9756271, 0.9756271, 0.9756271, 0.9756271, 0.9756271,
       0.9756271, 0.9756271, 0.9756271, 0.9756271, 0.9756271, 0.9756271,
       0.9756271, 0.9756271, 0.9756271, 0.9756271, 0.9756271, 0.9756271,
       0.9756271, 0.9756271, 0.9756271, 0.9756271, 0.9756271, 0.9756271,
       0.9756271, 0.9756271, 0.9756271, 0.9756271, 0.9756271, 0.9756271,
       0.9756271, 0.9756271, 0.9756271, 0.9756271, 0.9756271, 0.9756271,
       0.9756271, 0.9756271, 0.9756271, 0.9756271, 0.9756271, 0.9756271,
       0.9756271, 0.9756271, 0.9756271, 0.9756271, 

In [22]:
monte.start()


###### Start ######

[1.13875940e-03 1.14715082e-03 2.91497148e-03 4.98963335e-03
 5.46736557e-03 2.17216953e-03 3.32046583e-03 2.71074943e-03
 6.37633618e-03 2.89058033e-03 3.75598399e-03 5.88786289e-03
 6.01757829e-03 4.83503962e-03 5.58562914e-04 7.72934534e-03
 7.64418902e-03 7.71782384e-03 4.50669262e-03 4.83737818e-03
 2.99484270e-03 7.62730451e-03 6.73218459e-03 1.39322089e-03
 7.85536493e-03 6.88728959e-03 9.12020922e-03 5.12957168e-03
 6.93326705e-03 7.90240406e-04 9.34807655e-03 5.19826393e-03
 2.64897239e-03 7.54196718e-03 8.05627022e-03 7.51526838e-04
 3.27874498e-03 9.18903249e-03 1.39254134e-03 4.94128274e-03
 2.17793741e-03 2.83652127e-04 6.80611121e-03 1.49254130e-03
 2.76146939e-03 4.12874647e-03 3.22303977e-03 9.31470094e-03
 3.46125290e-03 7.17531728e-03 4.73277934e-03 6.57560319e-03
 9.34570487e-04 8.49971594e-05 5.66650655e-03 8.59878923e-04
 1.26984658e-03 2.60517609e-03 8.13111474e-03 1.87450711e-04
 6.79256190e-03 4.84280453e-03 9.81827596e-03 1.49768397e-03
 9

ValueError: operands could not be broadcast together with shapes (100,) (48,) 

In [ ]:
import cython
import numpy as np
%load_ext Cython

In [91]:
%%cython
cimport numpy as cnp
cnp.import_array()
#SINGLE = cnp.float32
#SHORT = cnp.int16
ctypedef cnp.float32_t SINGLE_t
ctypedef cnp.int16_t SHORT_t
def borderAct(
    cnp.ndarray[ndim=3, dtype = SINGLE_t] v,
    cnp.ndarray[ndim=3, dtype = SINGLE_t] p,
    cnp.ndarray[ndim=3, dtype = SHORT_t] add,
    cnp.ndarray[ndim=3, dtype = SINGLE_t] db,
    float l):
    
    cdef int array_size = v[0].size
    v = v.T
    p = p.T
    add = add.T
    db = db.T
    for i in xrange(array_size):
        
        v[i],p[i],add[i] = RTInterface(
            <float *> v[i].data,
            <float *>p[i],
            <float *>add[i],
            <int *>db[i],
            <float> l)
        
cdef RTInterface(
    float *v,float *p,int *add,float *db,float l):
    n = 0
    min_db = min(db)
    for i in xrange(3):
        if min_db == db[i]:
            n = i
    return float v,float p, int add        
    


Error compiling Cython file:
------------------------------------------------------------
...
    n = 0
    min_db = min(db)
    for i in xrange(3):
        if min_db == db[i]:
            n = i
    return float v,float p, int add        
                ^
------------------------------------------------------------

/Users/kaname/.ipython/cython/_cython_magic_be205662a021167b8e511332aadf5f43.pyx:35:17: Syntax error in simple statement list


In [80]:
a = [1,2,-1,0]
min_a = min(a)
n = 0
for i in range(len(a)):
    if min_a == a[i]:
        n = i
n

2

In [62]:
def borderAct(v,p,add,db,l):
    
    ind = np.argmin(db,axis=0)
    A = self.create01Array(ind,m=3)
    B = ~A
    ni = self.model.getReflectiveIndex(add)
    sigAv = A*np.sign(v)
    ds = (l/2-p[ind])/abs(v[ind])
    db -= ds
    p = sigAv*l/2+B*(p+v*ds)
    nt = self.model.getReflectiveIndex(add+sigAv)
    pb_index = np.where(ni!=nt)[0]

    if list(pb_index) != []:
        #透過判別
        #入射角の計算
        ai = np.arccos(abs(v[ind,pb_index]))
        sub_index = np.where((ai>np.arcsin(nt[pb_index]/ni[pb_index])))[0]

        #屈折角の計算
        ##(ai,at)=(0,0)の時は透過させ方向ベクトルは変化させない
        at = np.arcsin(ni[pb_index]*np.sin(ai)/nt[pb_index])
        Ra = np.random.rand(ai.size)\
        -((np.sin(ai-at)/np.sin(ai+at))**2+(np.tan(ai-at)/np.tan(ai+at))**2)/2
        #全反射はRaを強制的に0以下にし反射させる
        Ra[sub_index] = -1
        #透過と反射のindexを取得
        vl_index = pb_index[np.where(Ra<=0)[0]]#反射
        sub_index = np.where(Ra>0)[0]
        vt_index = pb_index[sub_index]#透過
        at = at[sub_index]
        v[:,vl_index] = B[:,vl_index]*v[:,vl_index]-A[:,vl_index]*v[:,vl_index]
        v[:,vt_index] = (ni[vt_index]/nt[vt_index])*B[:,vt_index]*v[:,vt_index]\
        +np.sign(sigAv[:,vt_index])*np.cos(at)
        add[:,vt_index] = add[:,vt_index] +sigAv[:,vt_index]
        p[:,vt_index] = -sigAv*l/2
    return v,p,add,db


array([[False,  True, False,  True, False,  True, False, False],
       [False,  True, False,  True, False,  True, False, False],
       [False,  True, False,  True, False,  True,  True,  True]])

In [63]:
%timeit ~a

388 ns ± 17 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [60]:
a = np.array([[1,0,1,0,1,0,1,1],[1,0,1,0,1,0,1,1],[1,0,1,0,1,0,1,1]]).astype(bool)
%timeit 1-a

1.8 µs ± 24.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [52]:
%timeit np.ones_like(a)-a

TypeError: numpy boolean subtract, the `-` operator, is not supported, use the bitwise_xor, the `^` operator, or the logical_xor function instead.

In [19]:
import cython
import numpy as np
%load_ext Cython
a = np.array([1,2,3])
b = np.array([1,2,3])


In [41]:
%%cython
cimport numpy as cnp
cpdef cy_fib(cnp.ndarray[ndim=1, dtype = cnp.int64_t] a,
           cnp.ndarray[ndim=1, dtype = cnp.int64_t] b):
    c = a*b
    #c = [i*j for i,j in zip(a,b)]
    return c

In [42]:
cy_fib(a,b)

array([1, 4, 9])

In [43]:
%timeit cy_fib(a,b)

2.33 µs ± 38.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [40]:
%timeit a*b

464 ns ± 10.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [16]:
model = boxcelPlateModel()
model.builtModel(thickness = [1,1],xy_size=10,space = 0.1,ma=[0.5,0.5],ms=[100,100],g=[0.9,0.8],n_list=[1.3,1.34],n_air=1)

0 Mbyte


In [17]:
model.xy_size

10